# Setup imports

In [1]:
import logging
import os
import random
import sys

import numpy as np
import pandas as pd
import torch
torch.cuda.empty_cache()

from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import DataLoader, ImageDataset
from monai.transforms import AddChannel, Compose, RandRotate90
from monai.transforms import Resize, ScaleIntensity, EnsureType

from datetime import datetime
from tqdm import tqdm

from constants import Constants
from patient import Patient

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

constants = Constants()

In [2]:
LABELS = constants.labels_pkl

LOGS = constants.logs
if not os.path.exists(LOGS): os.mkdir(LOGS)
    
BEST_METRICS = constants.best_metrics
if not os.path.exists(BEST_METRICS): os.mkdir(BEST_METRICS)

## Initialize log saving 

In [3]:
def log_write(command, file):
    file.write(f'{command}\n')
    print(command)

# Get existing data

In [4]:
def get_patients(labels):
    patient_data = {}
    for c, prox_id in enumerate(labels.ProxID):
        labels_row = labels.iloc[c]
        
        new_patient = Patient()
        new_patient.add_labels(labels_row)
        
        patient_data[prox_id] = new_patient
        
    return patient_data

In [5]:
labels_df = pd.read_pickle(LABELS)
patient_data = get_patients(labels_df)

# Split the dataset randomly

In [6]:
patients = list(labels_df.ProxID)

TRAIN_TEST_RATIO = constants.split_ratio
train_num = int(len(patients) * TRAIN_TEST_RATIO)

random.shuffle(patients)

train_data, test_data = patients[:train_num], patients[train_num:] 

len(train_data), len(test_data)

(163, 41)

# Prepare training data 

In [7]:
def get_lesion_summary(ClinSig):
    summary = False
    for cs in ClinSig:
        summary = summary or cs
    return summary


def get_images_and_labels(data, labels):
    images_arr = []
    labels_arr = []
    num_files_idx = []

    for patient_id in data:
        patient = patient_data[patient_id]

        label = 0
        clin_sig = get_lesion_summary(patient.ClinSig)
        if clin_sig: label = 1

        for t2 in patient.T2: 
            images_arr.append(t2)
            labels_arr.append(label)
        for adc in patient.ADC: 
            images_arr.append(adc)
            labels_arr.append(label)
        for dwi in patient.DWI: 
            images_arr.append(dwi)
            labels_arr.append(label)
        images_arr.append(patient.KTrans)
        labels_arr.append(label)
        num_files_idx.append(len(images_arr))
    return images_arr, labels_arr, num_files_idx

In [8]:
train_images_arr, labels_arr, num_files_idx = get_images_and_labels(train_data, labels_df)

train_labels_arr = np.array(labels_arr)
train_labels_arr = torch.nn.functional.one_hot(torch.as_tensor(labels_arr)).float()

len(train_images_arr), len(train_labels_arr), train_labels_arr.shape

(1029, 1029, torch.Size([1029, 2]))

## Define transforms

In [9]:
RESIZE_SIZE = constants.image_resize

train_transforms = Compose([
    ScaleIntensity(), 
    AddChannel(), 
    Resize((RESIZE_SIZE, RESIZE_SIZE, RESIZE_SIZE)), 
    RandRotate90(), 
    EnsureType()
])

val_transforms = Compose([
    ScaleIntensity(), 
    AddChannel(), 
    Resize((RESIZE_SIZE, RESIZE_SIZE, RESIZE_SIZE)), 
    EnsureType()
])

## Check loaders

In [10]:
BATCH_SIZE = constants.batch_size
NUM_WORKERS = constants.num_workers

# Define nifti dataset, data loader
check_ds = ImageDataset(
    image_files=train_images_arr, 
    labels=train_labels_arr, 
    transform=train_transforms
)
check_loader = DataLoader(
    check_ds, 
    batch_size=BATCH_SIZE, 
    num_workers=NUM_WORKERS, 
    pin_memory=pin_memory
)

im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label, label.shape)

<class 'torch.Tensor'> torch.Size([2, 1, 168, 168, 168]) tensor([[1., 0.],
        [1., 0.]]) torch.Size([2, 2])


## Create data loaders 

In [11]:
TRAIN_VAL_RATIO = constants.split_ratio
train_val_num = num_files_idx[int(len(train_data) * TRAIN_VAL_RATIO)] - 1 

train_images, val_images = train_images_arr[:train_val_num], train_images_arr[train_val_num:]
train_labels, val_labels = train_labels_arr[:train_val_num], train_labels_arr[train_val_num:]

# create a training data loader
train_ds = ImageDataset(
    image_files=train_images, 
    labels=train_labels, 
    transform=train_transforms
)
train_loader = DataLoader(
    train_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=NUM_WORKERS, 
    pin_memory=pin_memory
)

# create a validation data loader
val_ds = ImageDataset(
    image_files=val_images, 
    labels=val_labels, 
    transform=val_transforms
)
val_loader = DataLoader(
    val_ds, 
    batch_size=BATCH_SIZE, 
    num_workers=NUM_WORKERS, 
    pin_memory=pin_memory
)

len(train_loader), len(val_loader)

(412, 103)

# Train the data

## Define network, loss function, and optimizer

In [12]:
today = datetime.today()
date_format_metric = today.strftime("%Y%m%d_%H%M%S")
print(date_format_metric)

model = constants.model.to(device)
loss_function = constants.loss_function
optimizer = constants.optimizer

20220719_171020


## Training parameters

In [13]:
EPOCHS = constants.epochs
MODEL_TYPE = constants.model_type
LOSS_TYPE = constants.loss_type
OPTIMIZER_TYPE = constants.optimizer_type
LR = constants.lr

val_interval = 1
best_metric = -1
best_metric_epoch = -1

epoch_loss_values = []
metric_values = []
writer = SummaryWriter()

## Start training 

In [14]:
log_filename_train = LOGS / f'{date_format_metric}_E{EPOCHS}_train.txt'
log_filename_test = LOGS / f'{date_format_metric}_E{EPOCHS}_test.txt'

In [15]:
log_file = open(log_filename_train, 'w')

log_write(f'Model         :: {MODEL_TYPE}', log_file)
log_write(f'Loss function :: {LOSS_TYPE}', log_file)
log_write(f'Optimizer     :: {OPTIMIZER_TYPE}', log_file)
log_write(f'Learning rate :: {LR}\n', log_file)

for epoch in range(EPOCHS):
    log_write("-" * 10, log_file)
    log_write(f"Epoch {epoch + 1}/{EPOCHS}", log_file)
    model.train()
    epoch_loss = 0
    step = 0
    
    MODEL_STATE_DICT = BEST_METRICS / f'{date_format_metric}.pth'
    
    for batch_data in tqdm(train_loader):
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    log_write(f"Epoch {epoch + 1} average loss: {epoch_loss:.4f}", log_file)

    if (epoch + 1) % val_interval == 0:
        model.eval()

        num_correct, metric_count = 0.0, 0
        ones, zeros = 0, 0
        
        for val_data in val_loader:
            val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
            with torch.no_grad():
                val_outputs = model(val_images).argmax(dim=1)
                val_labels = val_labels.argmax(dim=1)
                value = torch.eq(val_outputs, val_labels)
                metric_count += len(value)
                num_correct += value.sum().item()

        metric = num_correct / metric_count
        metric_values.append(metric)

        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), MODEL_STATE_DICT)
            log_write("Saved new best metric model", log_file)

        log_write(f"Current epoch: {epoch+1} current accuracy: {metric:.4f}", log_file)
        log_write(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}", log_file)
        writer.add_scalar("val_accuracy", metric, epoch + 1)

log_write(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}", log_file)
writer.close()
log_file.close()

Model         :: DensNet121(3, 1, 2)
Loss function :: Binary Cross Entropy
Optimizer     :: Adam
Learning rate :: 0.0001

----------
Epoch 1/50


100%|████████████████████████████████████████████████████████████████████████████████| 412/412 [04:55<00:00,  1.39it/s]


Epoch 1 average loss: 0.6724
Saved new best metric model
Current epoch: 1 current accuracy: 0.8098
Best accuracy: 0.8098 at epoch 1
----------
Epoch 2/50


100%|████████████████████████████████████████████████████████████████████████████████| 412/412 [04:47<00:00,  1.43it/s]


Epoch 2 average loss: 0.6652
Current epoch: 2 current accuracy: 0.7854
Best accuracy: 0.8098 at epoch 1
----------
Epoch 3/50


...

Epoch 48 average loss: 0.4938
Current epoch: 48 current accuracy: 0.7220
Best accuracy: 0.8146 at epoch 25
----------
Epoch 49/50


100%|████████████████████████████████████████████████████████████████████████████████| 412/412 [04:48<00:00,  1.43it/s]


Epoch 49 average loss: 0.4624
Current epoch: 49 current accuracy: 0.6634
Best accuracy: 0.8146 at epoch 25
----------
Epoch 50/50


100%|████████████████████████████████████████████████████████████████████████████████| 412/412 [04:47<00:00,  1.43it/s]


Epoch 50 average loss: 0.4530
Current epoch: 50 current accuracy: 0.7561
Best accuracy: 0.8146 at epoch 25
Training completed, best_metric: 0.8146 at epoch: 25


# Test the model

## Prepare test data 

In [16]:
test_images_arr, labels_arr, _ = get_images_and_labels(test_data, labels_df)

test_labels_arr = np.array(labels_arr)
test_labels_arr = torch.nn.functional.one_hot(torch.as_tensor(labels_arr)).float()

len(test_images_arr), len(test_labels_arr), test_labels_arr.shape

(260, 260, torch.Size([260, 2]))

## Create test loader

In [17]:
BATCH_SIZE_TEST = constants.batch_size_test
NUM_WORKERS_TEST = constants.num_workers

test_ds = ImageDataset(
    image_files=test_images_arr, 
    labels=test_labels_arr, 
    transform=val_transforms
)

test_loader = DataLoader(
    test_ds, 
    batch_size=BATCH_SIZE_TEST, 
    num_workers=NUM_WORKERS_TEST, 
    pin_memory=torch.cuda.is_available()
)

In [18]:
def calculate_KPI(kpi, output, label):
    tp, tn, fp, fn = kpi
    
    type_KPI = ''
    
    if label[0]:
        if output[0]: 
            tp += 1
            type_KPI = 'TP'
        else: 
            fn +=1
            type_KPI = 'FN'
    else:
        if output[0]: 
            fp += 1
            type_KPI = 'FP'
        else: 
            tn +=1
            type_KPI = 'TN'
    return (tp, tn, fp, fn), type_KPI


def get_metrics(kpi):
    tp, tn, fp, fn = kpi
    
    recall, precision, f1 = -1, -1, -1
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    if tp + fn != 0: 
        recall = tp / (tp+fn)
        
    if tp + fp != 0: 
        precision = tp / (tp+fp)
        
    if precision > 0 and recall > 0: 
        f1 = (2*precision*recall) / (precision+recall)
    
    return accuracy, recall, precision, f1

In [19]:
log_file = open(log_filename_test, 'w')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_test = constants.model.to(device)

MODEL = BEST_METRICS / f'{date_format_metric}.pth'

log_write(f'Training log :: {log_filename_train}', log_file)
log_write(f'Metrics      :: {MODEL}\n', log_file)
log_write("-" * 10, log_file)

model_test.load_state_dict(torch.load(MODEL))
model_test.eval()

fps, fns = [], []

with torch.no_grad():
    KPI = 0, 0, 0, 0
    
    for i, data in enumerate(tqdm(test_loader)):
        test_images, test_labels = data[0].to(device), data[1].to(device)
        
        test_outputs = model_test(test_images).argmax(dim=1)
        test_labels = test_labels.argmax(dim=1)
        
        KPI, type_KPI = calculate_KPI(KPI, test_outputs, test_labels)
        
        if type_KPI == 'FP':
            fps.append((i, test_labels, test_outputs))
        elif type_KPI == 'FN':
            fns.append((i, test_labels, test_outputs))
        
    log_write(f'(TP, TN, FP, FN) = {KPI}', log_file)
    
    metrics = get_metrics(KPI)
    log_write(
        f'Evaluation metrics:\n'
        f'Accuracy  : {metrics[0]}\n'
        f'Recall    : {metrics[1]}\n'
        f'Precision : {metrics[2]}\n'
        f'F1-score  : {metrics[3]}',
        log_file
    )
    
log_file.close()

Training log :: logs\20220719_171020_E50_train.txt
Metrics      :: best_metrics\20220719_171020.pth

----------


100%|████████████████████████████████████████████████████████████████████████████████| 260/260 [00:54<00:00,  4.78it/s]

(TP, TN, FP, FN) = (5, 146, 4, 105)
Evaluation metrics:
Accuracy  : 0.5807692307692308
Recall    : 0.045454545454545456
Precision : 0.5555555555555556
F1-score  : 0.08403361344537816
